Steps in loop
- Get image file name
- Get associated text (or texts)
- Compute embedding
- Save embedding to new file embeddings folder
- Make dataset class to handle the preprocessing

In [ ]:
import os
import numpy as np
from pathlib import Path
from imagen_pytorch.t5 import t5_encode_text, DEFAULT_T5_NAME
from typing import Any, Callable, Optional, Tuple, List
from PIL import Image
import itertools

import torchvision
from pycocotools.coco import COCO

In [ ]:
def get_emb_tensor(targets):
    text_embeds, text_masks = t5_encode_text(targets, name = TEXT_ENCODER_NAME)
    text_embeds = text_embeds.cpu().numpy()
    text_masks = text_masks.cpu().numpy()
    # text_embeds, text_masks = map(lambda t: t.to('cuda:0'), (text_embeds, text_masks))
    return text_embeds, text_masks

In [ ]:
root = "coco\\images\\train2014",
train_annFile = "coco\\annotations\\captions_train2014.json"
valid_annFile = "coco\\annotations\\captions_val2014.json"
train_embed_dir = "coco\\embeddings\\train2014\\"
valid_embed_dir = "coco\\embeddings\\val2014\\"
TEXT_ENCODER_NAME = "t5-large"

coco = COCO(train_annFile)
ids = list(sorted(coco.imgs.keys()))

In [4]:
def get_one(id_):
    path = coco.loadImgs(id_)[0]["file_name"]
    captions = coco.loadAnns(coco.getAnnIds(id_))
    caption = captions[0]['caption']
    text_embed, text_mask = get_emb_tensor(caption)
    return path, caption, text_embed, text_mask


def get_path_caption_lists():
    img_paths = []
    img_captions = []
    for id_ in ids:
        path = coco.loadImgs(id_)[0]["file_name"]
        targets = coco.loadAnns(coco.getAnnIds(id_))
        captions = [tar['caption'] for tar in targets]
        img_paths.append(path)
        img_captions.append(captions[0])
    return img_paths, img_captions

In [6]:
def save_embeddings(img_paths, img_captions, batch_size = 256):
    
    length = len(ids)

    for i in range(0, 512, batch_size):
        if i == ((length // batch_size) * batch_size):
            paths = img_paths[i:]
            captions = img_captions[i:]
            # text_embeds, text_masks = get_emb_tensor(captions)
        else:
            paths = img_paths[i:i+batch_size]
            captions = img_captions[i:i+batch_size]
            # text_embeds, text_masks = get_emb_tensor(captions)
        for i in range(0, len(paths)):
            img_path = paths[i]
            embed_fp = train_embed_dir + img_path[:-4] + "_embedding" + ".npy"
            mask_fp = train_embed_dir + img_path[:-4] + "_mask" + ".npy"
            # np.save(id_, )
            print(embed_fp, mask_fp)

In [ ]:
img_captions

In [ ]:
# img_paths

In [ ]:
def load_image(root, id):
    path = coco.loadImgs(id)[0]["file_name"]
    return Image.open(os.path.join(root, path)).convert("RGB")

def load_target(root, id):
    return coco.loadAnns(coco.getAnnIds(id))


def precompute_embeddings(root, annFile):
    # root = "coco\\images\\train2014"
    
    coco = COCO(annFile)
    ids = list(sorted(coco.imgs.keys()))

    # for id in ids
    # ... 